In [1]:
import csv
import subprocess
import io

import imageio
from IPython.display import HTML

import constants
import map_structure
import preprocess
import hivemind_segments_from_video

In [12]:
# Define the path to the video file and list of events with their timestamps
video_path = './game_637156.mkv'
events = [
    {'name': 'game start', 'time': 5.16},
    {'name': 'last gate tag', 'time': 160.608 + 5.16},
    {'name': 'Event 3', 'time': 135}
]

seek_js = """
<script>function seekVideo(timestamp) {
    var videoPlayer = document.getElementById('video-player');
    videoPlayer.currentTime = timestamp;
}
</script>
"""

# Define the HTML code for the video player and event buttons
video_player_html = f'<video id="video-player" src="{video_path}" controls width=740 height=405></video>'
event_buttons_html = ""
for event in events:
    event_buttons_html += f"""
    <button onclick="seekVideo({event['time']})">{event['name']}</button>
    """

# Combine the HTML and JavaScript code and display it
html = f"""
{seek_js}
{video_player_html}
<div>
{event_buttons_html}
</div>
"""
display(HTML(html))


In [3]:
video = imageio.get_reader(video_path, 'ffmpeg')

def is_start_screen(frame):
    texts = hivemind_segments_from_video.extract_text(frame)
    return hivemind_segments_from_video.is_kq_start_screen_ocr_text(texts)


def find_game_start_frame():
    found_start_screen = False
    step_size = 180
    for cand_frame in range(0, len(video), 180):
        img = video.get_data(cand_frame)
        if found_start_screen:
            if not is_start_screen(img):
                print('found non start screen at ', cand_frame)
                for maybe_last_start in range(cand_frame - step_size + 1, cand_frame):
                    img = video.get_data(maybe_last_start)
                    if not is_start_screen(img):
                        print('game start frame is', maybe_last_start, 'in seconds', maybe_last_start /60)
                        return maybe_last_start

        else:
            found_start_screen = is_start_screen(img)
            if found_start_screen:
                print('got first start screen at', cand_frame)
    return None
                
#start_frame = find_game_start_frame()
#print(start_frame)
start_frame = 310

In [9]:
def get_events_for_game(game_id):
    cmd_list = ['grep -h',  f"',{game_id}'", 
                'validated_all_gameevent_partitioned/gameevents_*.csv']
    raw_events_str = subprocess.getoutput(' '.join(cmd_list))
    csv_reader = csv.DictReader(
        io.StringIO(constants.GAME_EVENT_HEADER + '\n' + raw_events_str))
    
    game_event_iterator = preprocess.iterate_events_from_csv_reader(csv_reader)
    
    msi = map_structure.MapStructureInfos()
    yield from preprocess.iterate_game_events_with_state(game_event_iterator, msi)
    
for game_id, event, game_state, all_game_events in get_events_for_game(637156):
    #start_and_end_events = [e for e in all_game_events if type(e) in (preprocess.MapStartEvent, 
    #                                                                  preprocess.GameStartEvent)]
    preprocess.debug_print_events(all_game_events)
    break


Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 8, 'timestamp': -9.743}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 5, 'timestamp': -9.18}
MapStart {'game_id': 637156, 'game_version': '17.26', 'gold_on_left': False, 'map': <Map.map_day: 'map_day'>, 'timestamp': -8.984}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 1, 'timestamp': -8.984}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 2, 'timestamp': -8.984}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 9, 'timestamp': -8.864}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 4, 'timestamp': -8.826}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 10, 'timestamp': -8.764}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 3, 'timestamp': -8.466}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 6, 'timestamp': -8.328}
Spawn {'game_id': 637156, 'is_bot': False, 'position_id': 7, 'timestamp': -6.804}
BlessMaiden {'game_id': 637156, 'gate_color': <Co

In [5]:
#for game_id, event, game_state, all_game_events in get_events_for_game(637156):

#for x in :
#preprocess.debug_print_events(get_events_for_game(637156))